# SIMPLE KRIGING IN PYTHON - 1D Application for a Single Point

### Note:

*This Notebook is based on an original Post from Connor Johnson, the original post is found here:
http://connor-johnson.com/2014/03/20/simple-kriging-in-python/. It has been transferred to a Jupyter Notebook to make it more interactive.*

*Changes being made:*
+ *using a common variogram function*
+ *calculation of variance map* 
+ *some code structure*

*The example data is modified to represent a quasi-1D distribution (along the x-axis), to improve comprehensibility. Visualisation has been changed to consider this, and puts an additional focus on the uncertainty of the prediction*

*The Kriging Method has been further broken up into its elementary steps to allow better understanding and the facilitates further developments. Accordingly, this Notebook only estimates and visualises a single point.

# Original Post

In this post I will work through an example of Simple Kriging. Kriging is a set of techniques for interpolation. It differs from other interpolation techniques in that it sacrifices smoothness for the integrity of sampled points. Most interpolation techniques will over or undershoot the value of the function at sampled locations, but kriging honors those measurements and keeps them fixed. In future posts I would like to cover other types of kriging, other semivariaogram models, and colocated co-kriging. Until then, I’m keeping relatively up to date code at my GitHub project, geostatsmodels. (https://github.com/cjohnson318/geostatsmodels)

In [ ]:
from pylab import *
import numpy as np
from pandas import DataFrame, Series
from scipy.spatial.distance import pdist, squareform
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [16 / 1.5, 10 / 1.5]   # inch / cm = 2.54
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'
# plt.rcParams['savefig.frameon'] = False

## Loading the Data

The data used in this exercise is in a zip file at this site. (Click on Geostatistics Resources.) I have adapted my material from the Kriging document on the same site. My approach will focus more on programming. First we will import some modules and then load the data and parse it,

In [ ]:
zraw = open( 'example_data/ZoneA.dat','r' ).readlines()
zraw = [ i.strip().split() for i in zraw[10:] ]
zraw = np.array( zraw, dtype=np.float )
zraw = DataFrame( zraw, columns=['x','y','thk','por','perm','lperm','lpermp','lpermr'] )

### Choose a sub-set and reduce to Quasi-1D

For this exercise, we need to reduce the data, and only choose values within a certain intervall for y, and create a quasi-1D Dataset by zeroing the y-coordinate. This is necessary as the method may fail during matrix inversion if there are two points with (near) equal x-coordinates, leading to a singular matrix.

In [ ]:
# choose the data subset from a y-intervall
ymean = 7500
yinterval = 2000

z = zraw  # create copy of full data set

# show the intervall before reducing the data
fig, ax = subplots()
ax.scatter( z.x, z.y, c=z.por, cmap='gray' )
ax.plot((-1500,22000),(ymean - yinterval / 2, ymean - yinterval / 2), "r--")
ax.plot((-1500,22000),(ymean + yinterval / 2, ymean + yinterval / 2), "r--")
ax.plot((-1500,22000),(ymean, ymean), "r-")
ax.set_aspect(1)
xlim(-1500,22000)
ylim(-1500,17500)
xlabel('Easting [m]')
ylabel('Northing [m]')
title('Porosity %') ;

plt.show()

z = z.loc[z.y > ymean - yinterval / 2]
z = z.loc[z.y < ymean + yinterval / 2]


In [ ]:
# reduce to quasi 1-D by assigning zero as y-coordinates for all samples,
# and sort the array by x-coordinate for better plottability.

z.y = 0.
z = z.sort_values(by="x")

plt.subplot(211)
plt.scatter( z.x, z.y, c=z.por, cmap='gray' )
ylabel('Northing [m]')
title('Porosity %') ;

plt.subplot(212)
plt.plot(z.x, z.por, ".--")
ylabel('Value')
xlabel('Easting [m]')
plt.show()

# part of our data set recording porosity
P = np.array( z[['x','y','por']] )
print("P =")
print(P)


## The Semivariogram

The semivariogram encodes data about spatial variance over the region at a given distance or lag. We generally expect data points that are close together spatially to share other characteristics, and we expect points that are separated by greater distances to have lesser correlation. The semivariogram allows us to model the similarity points in a field as a function of distance. The semivariogram is given by,

$$\begin{equation*} \hat{\gamma}(h) = \dfrac{1}{2N(h)} \displaystyle \sum_{N(h)} ( z_{i} - z_{j} )^{2} \end{equation*}$$ (Equation 1)

Here, h is distance specified by the user, and z_{i} and z_{j} are two points that are separated spatially by h. The N(h) term is the number of points we have that are separated by the distance h. The semivariogram then is the sum of squared differences between values separated by a distance h. As an aside, contrast this with the formulation for variance,

$$\begin{equation*} s = \dfrac{1}{N-1} \displaystyle \sum_{k=1}^{N} (z_{k} - \hat{\mu} )^{2} \end{equation*}$$ (Equation 2)

Here, $N$ is the number of data points, $\hat{\mu}$ is the sample mean, and $z_{k}$ is a data point. For sample variance, we are taking the squared difference between data points and the mean, and in the semivariogram we are taking the squared difference between data points separated by distance h. We can write some functions to calculate the semivariogram at one lag, and then at multiple lags as follows.

In [ ]:
def SVh( P, h, bw ):
    '''
    Experimental semivariogram for a single lag
    '''
    pd = squareform( pdist( P[:,:2] ) )
    N = pd.shape[0]
    Z = list()
    for i in range(N):
        for j in range(i+1,N):
            if( pd[i,j] >= h-bw )and( pd[i,j] <= h+bw ):
                Z.append( ( P[i,2] - P[j,2] )**2.0 )
    return np.sum( Z ) / ( 2.0 * len( Z ) )
 
def SV( P, hs, bw ):
    '''
    Experimental variogram for a collection of lags
    '''
    sv = list()
    for h in hs:
        sv.append( SVh( P, h, bw ) )
    sv = [ [ hs[i], sv[i] ] for i in range( len( hs ) ) if sv[i] > 0 ]
    return np.array( sv ).T
 
def C( P, h, bw ):
    '''
    Calculate the sill
    '''
    c0 = np.var( P[:,2] )
    if h == 0:
        return c0
    return c0 - SVh( P, h, bw )

The C() function is the covariance function, and will be used later. Let us now calculate and plot the semivariogram,

In [ ]:
# bandwidth, plus or minus 250 meters
bw = 500
# lags in 500 meter increments from zero to 10,000
hs = np.arange(0,10500,bw)
sv = SV( P, hs, bw )
plot( sv[0], sv[1], '.-' )
xlabel('Lag [m]')
ylabel('Semivariance')
title('Empirical Semivariogram') ;
savefig('sample_semivariogram.png', fmt='png', dpi=200)

Modeling
Now that we’ve calculated the semivariogram, we will need to fit a model to the data. There are three popular models, the spherical, exponential, and the Gaussian. Here, we’ll implement the spherical model. First, we will present a function named opt() for determining the optimal value a for the spherical model.

In [ ]:
def opt( fct, x, y, C0, parameterRange=None, meshSize=1000 ):
    if parameterRange == None:
        parameterRange = [ x[1], x[-1] ]
    mse = np.zeros( meshSize )
    a = np.linspace( parameterRange[0], parameterRange[1], meshSize )
    for i in range( meshSize ):
        mse[i] = np.mean( ( y - fct( x, a[i], C0 ) )**2.0 )
    return a[ mse.argmin() ]

The opt() function finds the optimal parameter for fitting a spherical model to the semivariogram data. The spherical model is given by the function spherical(). On the last line we see that spherical() returns itself in a map() function, which seems odd. The idea is that the input h can be a single float value, or list or NumPy array of floats. If h is a single value, then line 9 is called. If h is a list or an array (an iterable) then line 17 is called, which applies line 9 to each value of h.

In [ ]:
import svmodels

Next, cvmodel() fits a model to the semivariogram data and returns a covariance method named covfct().

In [ ]:
def cvmodel( P, model, hs, bw, Cn=None ):
    '''
    Input:  (P)      ndarray, data
            (model)  modeling function
                      - spherical
                      - exponential
                      - gaussian
            (hs)     distances
            (bw)     bandwidth
    Output: (covfct) function modeling the covariance
    '''
    if Cn is None:
        Cn = 0
    # calculate the semivariogram
    sv = SV( P, hs, bw )
    # calculate the sill
    C0 = C( P, hs[0], bw )
    # calculate the optimal parameters
    param = opt( model, sv[0], sv[1], C0 )
    # return a covariance function
    covfct = lambda h, a=param: C0 - list(model( h, a, C0, Cn=Cn ))
    return covfct

At this point we’ll plot our model and see if it represents our data well.

In [ ]:
sp = cvmodel( P, model=svmodels.spherical, hs=np.arange(0,10500,500), bw=500, Cn=0.1 )
C0 = C( P, hs[0], bw )
plot( sv[0], sv[1], '.-' )
plot( sv[0], C0 - sp( sv[0] ) ) ;
title('Spherical Model')
ylabel('Semivariance')
xlabel('Lag [m]')
savefig('semivariogram_model.png',fmt='png',dpi=200)


## Simple Kriging


Now that we have a model for the semivariogram, we can write a function to perform the kriging. The fundamental relationship is a matrix equation,

$$\begin{equation*} K \lambda = k \Rightarrow \lambda = K^{-1} k \end{equation*} $$ 

Here, $K$ is a matrix of covariances of sampled data points, calculated using the spherical model, $\lambda$ is a vector of simple kriging weights, and $k$ is the vector of covariances between the data points and an unsampled point. Our kriging function takes the data set $P$, the model, the distances hs, the bandwidth bw, the coordinates of the unsampled point u, and the number of surrounding points N to use in the calculation.

### Choosing the parameters

In [ ]:
# h, k, K, weights = krige( P, sp, (dy*j,dx*i), len(z) / 2 )       
# PARAMETER DEFINITION
#           (P)     data set [y,x,value]
#           (model) modeling function
#                    - spherical
#                    - exponential
#                    - gaussian
#           (u)     unsampled point
#           (N)     number of neighboring
#                   points to consider

# P - Data set already defined
covfct = sp  # covariance function
u = (7900, 0)  # (x,y) - coordinates of unsampled points
N = 5  # number of neighbors to consider
assert N < len(P) + 1, "Number of Neighbors greater than number of data points"

### Performing the Kriging Process for a single point

In [ ]:
# the mean of the variable
mu = np.mean( P[:,2] )
print("mu = " + str(mu))

The distance between the unsampled point $u$ and each data point in $P$ is calculated, creating a vector $d$ of size 1xM, where M is the number of data points.

This vector becomes an additional column to the matric $P$, which allows to sort the rows of $P$ by their distance to the data point, and only selecting to closest N neighbors.. 

In [ ]:
# distance between u and each data point in P (1xN vector)
d = np.sqrt( ( P[:,0]-u[0] )**2.0 + ( P[:,1]-u[1] )**2.0 )  # 2D 
print("d = "+str(d))

In [ ]:
# add these distances to P
P_n = np.vstack(( P.T, d )).T
# sort P by these distances
# take the first N of them
P_n = P_n[d.argsort()[:N]]
print("P_n = \n" + str(P_n))

For each of the remaining points, the expected covariance between the point and the unsampled point $u$ is calculated based on its distance by the convariance function

$$k = cov(d)$$

In [ ]:
# apply the covariance model to the distances
k = covfct( P_n[:,3] )
# cast as a matrix
k = np.matrix( k ).T
print("k =\n" + str(k))

In [ ]:
k

To create the covariance matrix $K$ with the covariances of all sampled points, the same function is being used:

$$K = [cov(d_{ij})]$$

with $d_{ij}$ being the Euclidian distance between points $i$ and $j$.

In [ ]:
# form a matrix of distances between existing data points
K = squareform( pdist( P_n[:,:2] ) )
# apply the covariance model to these distances
K = covfct( K.ravel() )
# re-cast as a NumPy array -- thanks M.L.
K = np.array( K )
# reshape into an array
K = K.reshape(N,N)
# cast as a matrix
K = np.matrix( K )
print("K =\n"+str(K))

In [ ]:
K

this allows calculating the Kriging weights $\lambda$ as defined previously

$$\lambda = K^{-1} k$$ 

In [ ]:
print("K-1 =\n"+str(np.linalg.inv( K )))

In [ ]:
# calculate the kriging weights
weights = np.linalg.inv( K ) * k
weights = np.array( weights )
print("weights =\n"+str(weights))

$$\epsilon = P[:,2] - \mu$$

In [ ]:
# calculate the residuals
residuals = P_n[:,2] - mu
print("residuals = "+str(residuals))

$$e(u) =\lambda^T \cdot \epsilon + \mu $$

In [ ]:
# calculate the estimation
estimation = np.dot( weights.T, residuals ) + mu
print("estimation = "+str(estimation))

$$\sigma^2 = C_0 - \lambda^T \cdot k$$

In [ ]:
variance = C0 - np.dot( weights.T, k )
print("variance = "+str(variance))

### Visualize the Result

The plot below shows the estimate (error bar equals +/- 1 standard deviation) and the sampled points. For the {{N}} samples closest to the estimate the kriging weights are shown.  

In [ ]:
plt.plot(z.x, z.por, 
         "o--", label="samples")
plt.scatter((u[0]), (estimation), 
            c="red", label="estimate")
plt.errorbar((u[0]), (estimation), yerr = sqrt(float(variance)), 
             fmt="r", label="standard deviation")

offset = 0.05
for i in range(len(weights)):
    plt.text(P_n[i,0]+offset, 
             P_n[i,2]+offset, 
             "{:.2f}".format(float(weights[i])))

ylabel('Value')
xlabel('Easting [m]')
plt.legend()
plt.show()